<a href="https://colab.research.google.com/github/GuiSSMartins/DTx-Project/blob/main/DTx_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc
from sklearn.metrics import roc_auc_score
import gc #importing garbage collector
import time
from scipy import signal


In [ ]:
df_train = pd.read_csv('/kaggle/input/m5-forecasting-uncertainty/sales_train_validation.csv')
df_prices = pd.read_csv('/kaggle/input/m5-forecasting-uncertainty/sell_prices.csv')
df_days = pd.read_csv('/kaggle/input/m5-forecasting-uncertainty/calendar.csv')

df_train = reduce_mem_usage(df_train)
df_prices = reduce_mem_usage(df_prices)
df_days = reduce_mem_usage(df_days)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')